# Walk-Forward CV Backtesting with Daily Rehedging

**Enhancements:**
1. Daily duration rehedging with rebalancing costs
2. Rolling window cross-validation for hyperparameter optimization
3. Proper train/validation/test split
4. Grid search over: entry threshold, exit threshold, stop loss, OU window

**Optimization Metric:** Sharpe Ratio

**Data Splits:**
- Train: 2015-2018 (find best hyperparameters)
- Validation: 2019 (select best config)
- Test: 2020-2024 (out-of-sample performance)

---

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Tuple, Dict, Optional, List
from itertools import product
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Create output directories
Path('../results/backtest_cv').mkdir(parents=True, exist_ok=True)
Path('../results/figures').mkdir(parents=True, exist_ok=True)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Reproducibility
np.random.seed(42)

print("✓ Setup complete")

✓ Setup complete


## 2. Configuration

In [ ]:
# ============================================================================
# BASE CONFIGURATION
# ============================================================================

BASE_CONFIG = {
    # Data splits
    'train_start': '2015-01-01',
    'train_end': '2018-12-31',
    'val_start': '2019-01-01',
    'val_end': '2019-12-31',
    'test_start': '2020-01-01',
    'test_end': '2024-12-31',
    
    # Position sizing
    'notional_capital': 1_000_000,
    'position_size': 0.5,  # 50% notional per leg
    
    # Transaction costs
    'lqd_spread_bps': 3,
    'ief_spread_bps': 2,
    
    # Duration (will be calculated dynamically)
    'rehedge_frequency': 1,  # Daily rehedging
    'rehedge_threshold_bps': 5,  # Only rehedge if duration drift > 5bps
    
    # Regime filter
    'trade_in_high_stress': False,
    'exit_on_regime_change': True,
    'vix_override_threshold': 30,  # Don't trade if VIX > 30
    
    # Trading rules
    'allow_overlapping_trades': False,
markdown
This notebook has been archived to `notebooks/archive_notebooks/04_walk_forward_cv_backtest.ipynb`.

If you need the original, open the archived file or recover it from git history.
Python 3
python
python3
python
3.10
markdown
This notebook has been archived to `notebooks/archive_notebooks/04_walk_forward_cv_backtest.ipynb`.

If you need the original, open the archived file or recover it from git history.
Python 3
python
python3
python
3.10
4
5



















































































































































































































































































































































































































































































































































































































































































































































































































































































































































5. Deploy to paper trading4. Add slippage model for large positions3. Consider adaptive thresholds based on regime2. Test on 2025 data when available1. Analyze which hyperparameters matter most### Next Steps   - Single position rule (no overlapping trades)   - VIX override (no entry if VIX > 30)   - Regime filtering (no trading in High Stress)4. **Enhanced Risk Controls:**   - Selected best config for final test   - Optimized for Sharpe ratio on validation set   - Grid search over entry/exit/stop thresholds + OU window3. **Hyperparameter Optimization:**   - Test: 2020-2024 (out-of-sample evaluation)   - Validation: 2019 (hyperparameter selection)   - Train: 2015-2018 (not used for CV)2. **Proper Train/Val/Test Split:**   - Includes transaction costs for rebalancing   - Rebalances IEF position when duration drift > 5 bps   - Recalculates hedge ratio daily based on current yields1. **Daily Duration Rehedging:**### Key Improvements Implemented## 14. Summarymarkdown#VSC-937a1bdbmarkdownprint("✓ Saved drawdown charts")plt.show()plt.savefig('../results/figures/cv_test_drawdowns.png', dpi=300, bbox_inches='tight')plt.tight_layout()ax2.grid(True, alpha=0.3)ax2.set_title('Classic Z-Score Drawdown', fontsize=13, fontweight='bold')ax2.set_xlabel('Date', fontsize=12)ax2.set_ylabel('Drawdown (%)', fontsize=12)ax2.plot(drawdown_classic.index, drawdown_classic * 100, color='coral', linewidth=1.5)ax2.fill_between(drawdown_classic.index, 0, drawdown_classic * 100, color='coral', alpha=0.3)drawdown_classic = (results_test_classic['equity'] - running_max_classic) / running_max_classicrunning_max_classic = results_test_classic['equity'].expanding().max()# Classicax1.grid(True, alpha=0.3)ax1.set_title('OU Process Drawdown', fontsize=13, fontweight='bold')ax1.set_ylabel('Drawdown (%)', fontsize=12)ax1.plot(drawdown_ou.index, drawdown_ou * 100, color='steelblue', linewidth=1.5)ax1.fill_between(drawdown_ou.index, 0, drawdown_ou * 100, color='steelblue', alpha=0.3)drawdown_ou = (results_test_ou['equity'] - running_max_ou) / running_max_ourunning_max_ou = results_test_ou['equity'].expanding().max()# OUfig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)# Plot drawdownspython#VSC-71c54382codeprint("✓ Saved equity curve")plt.show()plt.savefig('../results/figures/cv_test_equity_curves.png', dpi=300, bbox_inches='tight')plt.tight_layout()ax.grid(True, alpha=0.3)ax.legend()ax.set_title('Out-of-Sample Test: Optimized Strategies (2020-2024)', fontsize=14, fontweight='bold')ax.set_ylabel('Equity ($)', fontsize=12)ax.set_xlabel('Date', fontsize=12)ax.axhline(y=BASE_CONFIG['notional_capital'], color='gray', linestyle='--', alpha=0.5)        label='Classic Z-Score (Optimized)', linewidth=2, color='coral')ax.plot(results_test_classic.index, results_test_classic['equity'],         label='OU Process (Optimized)', linewidth=2, color='steelblue')ax.plot(results_test_ou.index, results_test_ou['equity'], fig, ax = plt.subplots(1, 1, figsize=(14, 6))# Plot equity curvespython#VSC-c0487952code## 13. Visualizationsmarkdown#VSC-96ca57e2markdownprint("\n✓ Results saved to results/backtest_cv/")print(comparison.to_string())print("=" * 100)print("FINAL COMPARISON: OUT-OF-SAMPLE TEST SET (2020-2024)")print("\n" + "=" * 100)comparison.to_csv('../results/backtest_cv/test_comparison.csv')})    'Classic Z-Score': metrics_test_classic    'OU Process': metrics_test_ou,comparison = pd.DataFrame({# Create comparison tableresults_test_classic.to_csv('../results/backtest_cv/test_classic_results.csv')results_test_ou.to_csv('../results/backtest_cv/test_ou_results.csv')# Save test resultspython#VSC-f396348ecode## 12. Save Results and Create Comparisonmarkdown#VSC-d876a400markdown    print(f"  {k:20s}: {v:.4f}")for k, v in metrics_test_classic.items():print("\nTest set performance:")metrics_test_classic = calculate_metrics(results_test_classic, BASE_CONFIG['notional_capital'])results_test_classic = engine.run(test_data, best_classic, method='classic', verbose=True)print(f"Parameters: {best_classic}")print("-" * 100)print("CLASSIC Z-SCORE - BEST CONFIG")print("\n" + "-" * 100)    print(f"  {k:20s}: {v:.4f}")for k, v in metrics_test_ou.items():print("\nTest set performance:")metrics_test_ou = calculate_metrics(results_test_ou, BASE_CONFIG['notional_capital'])results_test_ou = engine.run(test_data, best_ou, method='ou', verbose=True)print(f"Parameters: {best_ou}")print("-" * 100)print("OU PROCESS - BEST CONFIG")print("\n" + "-" * 100)print(f"Test observations: {len(test_data)}")print(f"\nTest period: {test_data.index.min().date()} to {test_data.index.max().date()}")print("=" * 100)print("OUT-OF-SAMPLE TEST: BEST CONFIGURATIONS")print("\n" + "=" * 100)best_classic = {k: best_params_classic[k] for k in param_keys}best_ou = {k: best_params_ou[k] for k in param_keys}param_keys = list(PARAM_GRID.keys())# Extract just the hyperparameters (not metrics)best_params_classic = top_configs_classic.iloc[0].to_dict()best_params_ou = top_configs_ou.iloc[0].to_dict()# Get best configspython#VSC-a138cde4code## 11. Test Best Configuration on Hold-Out Test Setmarkdown#VSC-e4dccce6markdownprint(f"\n✓ Saved top Classic configurations to results/backtest_cv/classic_top_configs.csv")top_configs_classic.to_csv('../results/backtest_cv/classic_top_configs.csv', index=False)# Save results)    top_n=5    method='classic',    PARAM_GRID,    val_data,    train_data,    engine,top_configs_classic = grid_search_cv(print("=" * 100)print("HYPERPARAMETER OPTIMIZATION: CLASSIC Z-SCORE")print("\n" + "=" * 100)python#VSC-16245d3ecode## 10. Run Grid Search for Classic Z-Scoremarkdown#VSC-b7da3342markdownprint(f"\n✓ Saved top OU configurations to results/backtest_cv/ou_top_configs.csv")top_configs_ou.to_csv('../results/backtest_cv/ou_top_configs.csv', index=False)# Save results)    top_n=5    method='ou',    PARAM_GRID,    val_data,    train_data,    engine,top_configs_ou = grid_search_cv(print("=" * 100)print("HYPERPARAMETER OPTIMIZATION: OU PROCESS")print("\n" + "=" * 100)# Run grid search for OU methodengine = WalkForwardBacktest(BASE_CONFIG)# Initialize enginepython#VSC-b6150084code## 9. Run Grid Search on Validation Setmarkdown#VSC-2cc69777markdownprint("✓ Grid search function defined")    return results_df.head(top_n)        print(results_df[display_cols].head(top_n).to_string(index=False))    display_cols = list(keys) + ['sharpe_ratio', 'annual_return', 'max_drawdown', 'n_trades']        print("=" * 100)    print(f"\nTop {top_n} configurations by Sharpe ratio:")    print(f"  Valid configurations: {len(results_df)}")    print(f"\n✓ Grid search complete")        results_df = results_df.sort_values('sharpe_ratio', ascending=False)    # Sort by Sharpe ratio        results_df = pd.DataFrame(results)                continue            print(f"\n  Error with params {params}: {e}")        except Exception as e:                        })                **metrics                **params,            results.append({            # Store                        )                engine.config['notional_capital']                backtest_results,            metrics = calculate_metrics(            # Calculate metrics                        )                verbose=False                method=method,                params,                val_data,            backtest_results = engine.run(            # Run backtest on validation data        try:                params = dict(zip(keys, combo))        # Create param dict    for combo in tqdm(combinations, desc="Grid search"):        results = []        print("\nRunning backtests...")    print(f"  Validation period: {val_data.index.min().date()} to {val_data.index.max().date()}")    print(f"  Total combinations: {len(combinations)}")        combinations = list(product(*values))    values = list(param_grid.values())    keys = list(param_grid.keys())    # Generate all combinations        print(f"  Parameter grid: {param_grid}")    print(f"Starting grid search for {method.upper()} method...")    """        Top N configurations sorted by Sharpe ratio    results : pd.DataFrame    -------    Returns            Number of top configurations to return    top_n : int        'ou' or 'classic'    method : str        Grid of hyperparameters to search    param_grid : dict        Validation data for hyperparameter selection    val_data : pd.DataFrame        Training data (not used for CV, but kept for consistency)    train_data : pd.DataFrame    engine : WalkForwardBacktest    ----------    Parameters        Grid search cross-validation for hyperparameter optimization.    """) -> pd.DataFrame:    top_n: int = 5    method: str = 'ou',    param_grid: dict,    val_data: pd.DataFrame,    train_data: pd.DataFrame,    engine: WalkForwardBacktest,def grid_search_cv(python#VSC-1f4c8522code## 8. Hyperparameter Optimization via Grid Searchmarkdown#VSC-9456653dmarkdownprint("✓ Metrics functions defined")    }        'win_rate': win_rate,        'n_trades': int(n_trades),        'total_return': total_return,        'calmar_ratio': calmar,        'max_drawdown': max_drawdown,        'annual_vol': annual_vol,        'annual_return': annual_return,        'sharpe_ratio': sharpe,    return {        win_rate = winning_days / total_days if total_days > 0 else 0    total_days = len(returns[returns != 0])    winning_days = (returns > 0).sum()    # Win rate        n_trades = (position_changes != 0).sum() // 2    position_changes = results['position'].diff().fillna(0)    # Trades        calmar = annual_return / abs(max_drawdown) if max_drawdown != 0 else 0    # Calmar        max_drawdown = drawdown.min()    drawdown = (equity_curve - running_max) / running_max    running_max = equity_curve.expanding().max()    # Max drawdown        sharpe = calculate_sharpe_ratio(results, initial_capital)    # Sharpe        annual_vol = returns.std() * np.sqrt(252)    # Volatility        annual_return = (1 + total_return) ** (1 / n_years) - 1 if n_years > 0 else 0    n_years = len(results) / 252    # Annualized return        total_return = (equity_curve.iloc[-1] - initial_capital) / initial_capital    # Total return        equity_curve = results['equity']    returns = results['pnl'] / initial_capital    """    Calculate comprehensive performance metrics.    """def calculate_metrics(results: pd.DataFrame, initial_capital: float) -> dict:    return annual_return / annual_vol if annual_vol > 0 else 0.0        annual_vol = returns.std() * np.sqrt(252)    annual_return = returns.mean() * 252            return 0.0    if len(returns) == 0 or returns.std() == 0:        returns = results['pnl'] / initial_capital    """    Calculate annualized Sharpe ratio.    """) -> float:    initial_capital: float    results: pd.DataFrame,def calculate_sharpe_ratio(python#VSC-e5afdec1code## 7. Performance Metricsmarkdown#VSC-b5ecb904markdownprint("✓ Enhanced backtest engine defined")        return results_df                    print(f"  Final equity: ${equity:,.2f}")            print(f"  Rehedge costs: ${self.rehedge_costs:,.2f}")            print(f"  Total costs: ${self.total_costs:,.2f}")            print(f"  Trades: {len([t for t in self.trades if t['action'] == 'ENTRY'])}")        if verbose:                results_df.set_index('date', inplace=True)        results_df = pd.DataFrame(results)                    })                **({'ou_' + k: v for k, v in ou_params.items()} if method == 'ou' else {})                'rehedge_costs': self.rehedge_costs,                'costs': self.total_costs,                'equity': equity,                'pnl': pnl,                'position': self.current_position,                'z_score': z_score,                'vix': current_vix,                'regime': current_regime,                'spread': current_spread,                'date': date,            results.append({            # Record                        equity += pnl            self.current_position = signal                            })                    'cost': cost                    'z_score': z_score,                    'signal': signal,                    'action': 'ENTRY' if signal != 0 else 'EXIT',                    'date': date,                self.trades.append({                # Record trade                                self.total_costs += cost                pnl -= cost                                    self.entry_date = None                    self.current_ief_notional = 0                    self.current_lqd_notional = 0                    # Clear position                                        )                        self.current_ief_notional                        self.current_lqd_notional,                    cost = self.calculate_transaction_cost(                    # Transaction cost                else:  # Closing position                                        self.entry_date = date                    self.current_ief_notional = ief_notional                    self.current_lqd_notional = lqd_notional                    # Update position                                        cost = self.calculate_transaction_cost(lqd_notional, ief_notional)                    # Transaction cost                                        ief_notional = lqd_notional * hedge_ratio                    lqd_notional = self.config['notional_capital'] * self.config['position_size']                    # Position sizing                                        hedge_ratio = self.calculate_hedge_ratio(current_yield)                    # Calculate hedge ratio                if signal != 0:  # Opening position            if position_change != 0:                        position_change = signal - self.current_position            # Execute trade if signal changes                            pnl -= rehedge_cost                pnl = self.calculate_pnl(self.current_position, spread_change, yield_change)                # Calculate P&L                                self.rehedge_costs += rehedge_cost                rehedge_cost = self.rehedge_position(target_hedge_ratio)                target_hedge_ratio = self.calculate_hedge_ratio(current_yield)                # Daily rehedging                                    yield_change = 0                else:                    yield_change = (current_yield - prev_yield) * 100                    prev_yield = data['dgs10'].iloc[i-1]                if 'dgs10' in data.columns:                                spread_change = current_spread - data['spread'].iloc[i-1]            if i > warmup and self.current_position != 0:                        rehedge_cost = 0            pnl = 0            # Calculate P&L from yesterday's position                                signal = 0  # Don't enter                if self.current_position == 0:            if current_vix > self.config['vix_override_threshold']:                                signal = 0                if self.config['exit_on_regime_change'] and self.current_position != 0:                    signal = 0                if not self.config['trade_in_high_stress'] and self.current_position == 0:            if current_regime == 1:  # High Stress            # Regime + VIX filter                        )                params['stop_loss_threshold']                params['exit_threshold'],                params['entry_threshold'],                self.current_position,                z_score,            signal = self.generate_signal(            # Generate signal                            ou_params = {}                )                    int(params['z_score_window'])                    current_spread,                    spread_history,                z_score = self.calculate_z_score_classic(            else:                )                    int(params['ou_train_window'])                    current_spread,                    spread_history,                z_score, ou_params = self.calculate_z_score_ou(            if method == 'ou':            # Calculate z-score                        spread_history = data['spread'].iloc[:i]                        current_vix = data['vix'].iloc[i]            current_yield = data['dgs10'].iloc[i]            current_regime = data['regime'].iloc[i]            current_spread = data['spread'].iloc[i]            date = data.index[i]        for i in range(warmup, len(data)):                    print(f"  Trading: {len(data) - warmup} days")            print(f"  Warm-up: {warmup} days")            print(f"  Params: {params}")            print(f"Running {method.upper()} backtest...")        if verbose:                    warmup = int(params['z_score_window'])        else:            warmup = int(params['ou_train_window'])        if method == 'ou':        # Warm-up (convert to int to avoid TypeError in range)                equity = self.config['notional_capital']        results = []                self.reset()        """        results : pd.DataFrame        -------        Returns                    Print progress        verbose : bool            'ou' or 'classic'        method : str            Hyperparameters (entry_threshold, exit_threshold, etc.)        params : dict            Must have: spread, regime, dgs10, vix        data : pd.DataFrame        ----------        Parameters                Run walk-forward backtest with daily rehedging.        """    ) -> pd.DataFrame:        verbose: bool = False        method: str = 'ou',        params: dict,        data: pd.DataFrame,        self,    def run(            return lqd_cost + ief_cost                ief_cost = ief_notional * (self.config['ief_spread_bps'] / 10000)        lqd_cost = lqd_notional * (self.config['lqd_spread_bps'] / 10000)        """        Calculate transaction cost for opening/closing position.        """    ) -> float:        ief_notional: float        lqd_notional: float,        self,    def calculate_transaction_cost(            return pnl                    pnl = -spread_chg * self.current_lqd_notional        else:  # Short spread            pnl = spread_chg * self.current_lqd_notional        if signal == 1:  # Long spread        # Simplified P&L (spread-only, since duration-hedged)                # For simplicity, use notionals directly        # Get current durations (approximation - could cache)                yield_chg = yield_change / 10000        spread_chg = spread_change / 10000        # Convert to decimal                    return 0.0        if signal == 0:        """        Uses current notionals (which may have been rehedged).                Calculate P&L for duration-neutral position.        """    ) -> float:        yield_change: float        spread_change: float,        signal: int,        self,    def calculate_pnl(            return cost                self.current_ief_notional = target_ief        # Update IEF notional                cost = delta_ief * (self.config['ief_spread_bps'] / 10000)        # Transaction cost for rebalancing IEF                    return 0.0        if drift_bps < self.config['rehedge_threshold_bps']:        drift_bps = (delta_ief / self.current_lqd_notional) * 10000        # Only rehedge if drift exceeds threshold                delta_ief = abs(target_ief - self.current_ief_notional)        # Change in IEF notional                target_ief = self.current_lqd_notional * target_hedge_ratio        # Target IEF notional                    return 0.0        if self.current_position == 0:        """        Returns rehedging cost.                Rebalance IEF position to maintain duration neutrality.        """    ) -> float:        target_hedge_ratio: float        self,    def rehedge_position(            return lqd_dur / ief_dur                ief_dur = calculate_ief_duration(current_yield)        lqd_dur = calculate_lqd_duration(current_yield)        """        Returns IEF notional / LQD notional.                Calculate duration-neutral hedge ratio.        """    ) -> float:        current_yield: float        self,    def calculate_hedge_ratio(            return current_position                        return +1            elif z_score < -entry_thresh:                return -1            if z_score > entry_thresh:        if current_position == 0:        # Entry signals                        return current_position            else:                return 0            if abs(z_score) < exit_thresh:        if current_position != 0:        # Exit signals                    return 0        if abs(z_score) > stop_thresh and current_position != 0:        # Stop loss        """        Returns +1 (long spread), -1 (short spread), 0 (flat).                Generate trading signal.        """    ) -> int:        stop_thresh: float        exit_thresh: float,        entry_thresh: float,        current_position: int,        z_score: float,        self,    def generate_signal(            return z, ou_params                z = ou_z_score(current_spread, ou_params['mu'], ou_params['sigma'])        ou_params = estimate_ou_parameters(recent)        recent = spread_history.iloc[-train_window:]                    return 0.0, {}        if len(spread_history) < train_window:        """Calculate OU-adjusted z-score."""    ) -> Tuple[float, dict]:        train_window: int        current_spread: float,        spread_history: pd.Series,        self,    def calculate_z_score_ou(            return (current_spread - mean) / std if std > 0 else 0.0                std = recent.std()        mean = recent.mean()        recent = spread_history.iloc[-window:]                    return 0.0        if len(spread_history) < window:        """Calculate classic rolling z-score."""    ) -> float:        window: int        current_spread: float,        spread_history: pd.Series,        self,    def calculate_z_score_classic(            self.rehedge_costs = 0        self.total_costs = 0        self.cash = self.config['notional_capital']        self.entry_date = None        self.current_ief_notional = 0        self.current_lqd_notional = 0        self.current_position = 0        self.trades = []        """Reset backtest state."""    def reset(self):            self.reset()        self.config = config    def __init__(self, config: dict):        """    Walk-forward backtesting engine with daily duration rehedging.    """class WalkForwardBacktest:python#VSC-7b53265dcode## 6. Enhanced Backtest Engine with Daily Rehedgingmarkdown#VSC-4b4b01cbmarkdownprint("✓ OU process functions defined")    return (current_spread - mu) / sigma if sigma > 0 else 0.0    """Calculate OU-adjusted z-score."""def ou_z_score(current_spread: float, mu: float, sigma: float) -> float:    }        'r_squared': r_squared        'half_life': half_life,        'sigma': sigma,        'mu': mu,        'theta': theta,    return {        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0    ss_tot = np.sum((dX - np.mean(dX))**2)    ss_res = np.sum(epsilon**2)            half_life = np.inf    else:        half_life = np.log(2) / theta    if theta > 0:        sigma = np.std(epsilon) * np.sqrt(1 / dt)    mu = -alpha / beta if beta != 0 else spread.mean()    theta = -beta / dt    # OU parameters        epsilon = dX - (alpha + beta * X)        alpha, beta = beta_hat    beta_hat = np.linalg.lstsq(X_design, dX, rcond=None)[0]    X_design = np.column_stack([np.ones(len(X)), X])    # OLS: dX = alpha + beta*X + epsilon        dX = spread.diff().dropna().values    X = spread.values[:-1]    """        theta, mu, sigma, half_life, r_squared    dict    -------    Returns            Time step (1.0 for daily data)    dt : float        Credit spread time series    spread : pd.Series    ----------    Parameters        Estimate Ornstein-Uhlenbeck process parameters using OLS.    """) -> Dict[str, float]:    dt: float = 1.0    spread: pd.Series,def estimate_ou_parameters(python#VSC-a5bb34b8code## 5. OU Process Implementationmarkdown#VSC-53f53740markdown    print(f"10Y Yield = {y:.1f}%: LQD dur = {lqd_dur:.2f}, IEF dur = {ief_dur:.2f}, Hedge ratio = {hedge_ratio:.3f}")    hedge_ratio = lqd_dur / ief_dur    ief_dur = calculate_ief_duration(y)    lqd_dur = calculate_lqd_duration(y)for y in test_yields:test_yields = [2.0, 3.0, 4.0]  # %print("=" * 60)print("Testing duration calculation:")# Test duration calculation    return calculate_modified_duration(treasury_yield, maturity=7.5)    treasury_yield = dgs10_yield / 100    """    IEF is 7-10Y treasuries, use ~7.5 year maturity.        Estimate IEF duration using 10Y treasury yield.    """def calculate_ief_duration(dgs10_yield: float) -> float:    return calculate_modified_duration(corp_yield, maturity=8.5)    corp_yield = dgs10_yield / 100 + 0.01    # Add typical credit spread (assume ~100bps)    """    LQD has ~8.5 year average maturity.        Estimate LQD duration using 10Y treasury yield + spread.    """def calculate_lqd_duration(dgs10_yield: float) -> float:    return modified_dur        modified_dur = macaulay_dur / (1 + ytm)    macaulay_dur = ((1 + ytm) / ytm) * (1 - 1 / ((1 + ytm) ** maturity))    # Simplified duration formula            ytm = 0.03  # Default to 3% if unrealistic    if ytm <= 0 or ytm > 0.20:  # Sanity check    """        Modified duration in years    float    -------    Returns            Annual coupon rate (not used in simplified formula)    coupon_rate : float, optional        Time to maturity in years    maturity : float        Yield to maturity (decimal, e.g., 0.03 for 3%)    ytm : float    ----------    Parameters        ModDur ≈ (1 + y) / y × (1 - 1 / (1 + y)^T)    Simplified formula for ETFs (assuming semi-annual coupons):        Calculate modified duration.    """) -> float:    coupon_rate: float = None    maturity: float,    ytm: float,def calculate_modified_duration(python#VSC-79caad05code- IEF: ~7.5 year average maturity (7-10Y treasuries)- LQD: ~8.5 year average maturity, use 10Y yield as proxy**Simplification for ETFs:**$$\text{Modified Duration} \approx \frac{1 + y}{y} \left(1 - \frac{1}{(1+y)^T}\right)$$For a bond with yield $y$, coupon rate $c$, and maturity $T$:### Modified Duration Formula## 4. Duration Calculation Utilitiesmarkdown#VSC-fdef337dmarkdownprint(f"  Test:  {len(test_data)} obs ({test_data.index.min().date()} to {test_data.index.max().date()})")print(f"  Val:   {len(val_data)} obs ({val_data.index.min().date()} to {val_data.index.max().date()})")print(f"  Train: {len(train_data)} obs ({train_data.index.min().date()} to {train_data.index.max().date()})")print(f"\nData splits:")test_data = df.loc[BASE_CONFIG['test_start']:BASE_CONFIG['test_end']]val_data = df.loc[BASE_CONFIG['val_start']:BASE_CONFIG['val_end']]train_data = df.loc[BASE_CONFIG['train_start']:BASE_CONFIG['train_end']]# Split dataprint(f"\nColumns: {df.columns.tolist()}")print(f"Date range: {df.index.min().date()} to {df.index.max().date()}")print(f"Data loaded: {len(df)} observations")df = pd.read_csv('../data/processed/full_processed_data_hmm.csv', index_col=0, parse_dates=True)# Load processed data with HMM regimespython#VSC-f2cae3f9code## 3. Data Loadingmarkdown#VSC-db886545markdownprint(f"  Test:       {BASE_CONFIG['test_start']} to {BASE_CONFIG['test_end']}")
print(f"\nHyperparameter grid size: {np.prod([len(v) for v in PARAM_GRID.values()])} combinations")

Configuration loaded

Data splits:
  Train:      2015-01-01 to 2018-12-31
  Validation: 2019-01-01 to 2019-12-31
  Test:       2020-01-01 to 2024-12-31

Hyperparameter grid size: 432 combinations
